In [ ]:
import math

def calculate_feeder_rpm_metric(mass_tpd, bulk_density, feeder_diameter, control_pitch):
    """
    Calculate the operating RPM for a screw feeder in metric units.

    Parameters:
    - mass_tpd: Mass throughput in tonnes per day.
    - bulk_density: Bulk density of the material in kg/m³.
    - feeder_diameter: Diameter of the screw feeder in meters.
    - control_pitch: Control pitch (length of the final screw flight) in meters.

    Returns:
    - rpm: Required RPM to achieve the target throughput.
    - vol_flow_m3_hr: Required volumetric flow in m³/hr.
    - displacement_m3: Displacement per revolution in m³.
    """
    #Convert tonnes per day to kg per day
    mass_kg_day = mass_tpd*1000.0
    #Convert kg per day to kg per hour
    mass_kg_hr = mass_kg_day/24.0

    #Calculate the required volumetric flow in m3/hr
    vol_flow_m3_hr = mass_kg_hr / bulk_density

    #Calculate the cross-sectional area of the screw (in m2)
    area_m2 = (math.pi * (feeder_diameter ** 2)) / 4.0

    #Calculate displacement per revolution (m3) = area (m2)*control pitch (m)
    displacement_m3 = area_m2 * control_pitch

    #Calculate required RPM:
    #RPM = (volumetric flow in m3/hr) / (displacement per revolution in m3*60 min/hr)
    rpm = vol_flow_m3_hr / (displacement_m3 * 60.0)

    return rpm, vol_flow_m3_hr, displacement_m3

# Example usage:

# Design parameters
mass_tpd = 50.2            # tonnes per day
bulk_density = 400         # kg/m3
feeder_diameter = 0.254    # 10 inches ≈ 0.254 m
control_pitch = 0.1016     # 4 inches ≈ 0.1016 m

rpm, vol_flow, disp = calculate_feeder_rpm_metric(mass_tpd, bulk_density, feeder_diameter, control_pitch)

print(f"Target mass flow: {mass_tpd} tonnes/day")
print(f"Required volumetric flow: {vol_flow:.3g} m³/hr")
print(f"Displacement per revolution: {disp:.3g} m³")
print(f"Calculated operating speed: {rpm:.3g} RPM")


Target mass flow: 50.2 tonnes/day
Required volumetric flow: 5.23 m³/hr
Displacement per revolution: 0.00515 m³
Calculated operating speed: 16.9 RPM


In [ ]:
def design_annular_pump(mass_flow=200, operating_temp=850, density=1900,
                        gap_thickness=0.01, current_density=1e6, magnetic_field=0.5,
                        pump_efficiency=0.5):
    """
    This function estimates the pump energy requirement based on a simplified model
    that uses the Lorentz force concept:
      - Volumetric flow rate, Q = mass_flow / density
      - Lorentz force per unit volume, F = current_density * magnetic_field
      - Developed pressure, Δ = F * gap_thickness
      - Fluid (mechanical) power, P_fluid = Δ * Q
      - Electrical power required = P_fluid / pump_efficiency

    Parameters:
      mass_flow:       Mass flow rate in kg/s (default: 200 kg/s)
      operating_temp:  Operating temperature in °C (default: 850 °C) [not used explicitly here]
      density:         Density of molten salt in kg/m³ (default: 1900 kg/m³)
      gap_thickness:   Annular duct gap thickness in meters (default: 0.01 m)
      current_density: Applied current density in A/m² (default: 1e6 A/m²)
      magnetic_field:  Magnetic field strength in Tesla (default: 0.5 T)
      pump_efficiency: Overall pump efficiency (default: 0.5 for 50%)

    Returns:
      Q:                    Volumetric flow rate in m³/s
      developed_pressure:   Developed pressure in Pascals
      fluid_power:          Mechanical power imparted to the fluid (W)
      electrical_power:     Electrical power input required (W)
    """
    # Calculate volumetric flow rate: Q = mass_flow / density
    Q = mass_flow / density  # in m³/s

    # Calculate Lorentz force per unit volume: F = J * B
    F = current_density * magnetic_field  # in N/m³

    # Developed pressure  in the annular duct: Δ = F * gap_thickness
    developed_pressure = F * gap_thickness  # in Pa

    # Mechanical power required: P_fluid = Δ * Q
    fluid_power = developed_pressure * Q  # in Watts

    # Electrical power input
    electrical_power = fluid_power / pump_efficiency  # in Watts

    return Q, developed_pressure, fluid_power, electrical_power


Q, pressure, fluid_power, elec_power = design_annular_pump()
print("Design results for Annular Linear Induction Pump:")
print(f"Volumetric flow rate Q = {Q:.3g} m³/s")
print(f"Developed pressure = {pressure:.3g} Pa")
print(f"Fluid power (mechanical) = {fluid_power:.3g} W")
print(f"Electrical power required = {elec_power:.3g} W")


Design results for Annular Linear Induction Pump:
Volumetric flow rate Q = 0.105 m³/s
Developed pressure = 5e+03 Pa
Fluid power (mechanical) = 526 W
Electrical power required = 1.05e+03 W


In [ ]:
import math

def design_molten_salt_tank(
    mass_kg=2000,
    density_kg_m3=1900,
    salt_temp_C=850,
    tank_height_m=2.5,
    firebrick_bottom_thickness_m=0.762,  # 30 inch
    firebrick_side_thickness_m=0.254,   # 10 inch
    kaowool_side_thickness_m=0.254,     # 10 inch
    kaowool_roof_thickness_m=0.305,     # 12 inch
    heater_power_per_kg_W=1.5,
    include_wall_thickness=True
):
    """
    Calculate tank volume, internal diameter, wall thickness, and insulation layout for molten salt buffer tank with integrated heating.
    """
    # STEP 1: Calculate required molten salt volume
    required_volume_m3 = mass_kg / density_kg_m3

    # STEP 2: Calculate internal diameter based on height
    tank_diameter_internal_m = math.sqrt((4 * required_volume_m3) / (math.pi * tank_height_m))

    # STEP 3: Estimate total external diameter including insulation layers
    total_side_insulation_thickness_m = firebrick_side_thickness_m + kaowool_side_thickness_m
    tank_diameter_external_m = tank_diameter_internal_m + 2 * total_side_insulation_thickness_m

    # STEP 4: Estimate total height including roof insulation
    total_height_m = tank_height_m + kaowool_roof_thickness_m

    # STEP 5: Heater power estimate
    heater_power_W = mass_kg * heater_power_per_kg_W

    # STEP 6: Add fixed wall thickness from empirical method
    if include_wall_thickness:
        carbon_steel_wall_thickness_m = 0.0254  # 25.4 mm (1 inch)
        ss316_liner_thickness_m = 0.00635       # 6.35 mm (1/4 inch)
    else:
        carbon_steel_wall_thickness_m = None
        ss316_liner_thickness_m = None

    # Output summary
    return {
        "Molten Salt Volume (m³)": round(required_volume_m3, 3),
        "Internal Tank Diameter (m)": round(tank_diameter_internal_m, 2),
        "External Tank Diameter (m)": round(tank_diameter_external_m, 2),
        "Total Height with Insulation (m)": round(total_height_m, 2),
        "Recommended Heater Power (W)": round(heater_power_W),
        "Carbon Steel Wall Thickness (m)": carbon_steel_wall_thickness_m,
        "SS316 Inner Liner Thickness (m)": ss316_liner_thickness_m
    }


# Example Usage
if __name__ == "__main__":
    design_results = design_molten_salt_tank()
    print("Molten Salt Tank Design with Insulation:")
    for key, value in design_results.items():
        print(f"- {key}: {value}")


Molten Salt Tank Design with Insulation:
- Molten Salt Volume (m³): 1.053
- Internal Tank Diameter (m): 0.73
- External Tank Diameter (m): 1.75
- Total Height with Insulation (m): 2.81
- Recommended Heater Power (W): 3000
- Carbon Steel Wall Thickness (m): 0.0254
- SS316 Inner Liner Thickness (m): 0.00635
